# Reconstruct the SBML from a set of reactions

This is to test the FBA toolchain

In [1]:
import sys
import os
import copy
import PyFBA
import re
import pickle

import inspect
inspect.getfile(PyFBA)

We are using /home/redwards/.local/lib/python3.9/site-packages/PyFBA-2.0-py3.9.egg/PyFBA/Biochemistry/ModelSEEDDatabase for our data


'/home/redwards/GitHubsLinux/PyFBA/venv/lib/python3.9/site-packages/PyFBA-2.1-py3.9.egg/PyFBA/__init__.py'

# Read the Reactions, Enzymes, and Compounds Sets

This parses the ModelSeed data

In [2]:
modelseed = PyFBA.parse.model_seed.parse_model_seed_data('gramnegative')
print(f"There are {len(modelseed.compounds):,} compounds, {len(modelseed.reactions):,} reactions, and {len(modelseed.enzymes):,} enzymes in total")

AttributeError: 'Path' object has no attribute 'open_text'

In [3]:
for c in modelseed.compounds:
    if '_' in c.name:
        print(c)
    elif 'cpd' in c.name:
        print(c)

cpd16266: M_Isobutyryl_CoA
cpd16272: M_trehalose_methoxy_mycolate_phosphate_2_cyclopropanated_rings
cpd29896: C02325_biomass
cpd29760: D-Xylose_biomass
cpd29816: L-Tyrosine_biomass
cpd29750: ATP_biomass
cpd16255: M_trehalose_keto_mycolate__1_cyclopropanated_rings
cpd29828: beta-D-Fructose_biomass
cpd29685: cdpdag_BS
cpd29885: C00062_biomass
cpd29641: an_alkane
cpd22268: ALPHA_ACETYLGLUCOSAMINIDES
cpd11448: lipo4-24_BS
cpd16248: M_PPM_methoxy_mycolate__2_cyclopropanated_rings
cpd29936: C05306_biomass
cpd29938: C00378_biomass
cpd16092: Phosphatidyllysine_SA2
cpd29823: Starch_biomass
cpd29749: 4-Coumaryl alcohol_biomass
cpd29809: L-Lysine_biomass
cpd29899: C00249_biomass
cpd16089: Phosphatidylleucine_SA
cpd29827: Ubiquinone_biomass
cpd29803: L-Cysteine_biomass
cpd11446: lipo2-24_BS
cpd11447: lipo3-24_BS
cpd16087: Phosphatidylglycine_SA2
cpd29902: C00458_biomass
cpd29756: Cellulose_biomass
cpd29907: C00075_biomass
cpd11442: gtca1-45_BS
cpd29795: GTP_biomass
cpd16253: M_trehalose_methoxy_my

In [4]:
rcpd = set()
for r in modelseed.reactions:
    for c in modelseed.reactions[r].all_compounds():
        if c.id == 'cpd00060':
            rcpd.add(str(c))
print("\n".join(rcpd))

cpd00060: L-Methionine (location: c)
cpd00060: L-Methionine (location: e)


In [44]:
for c in modelseed.compounds:
    if c.name == 'Fe2+':
        print(c)

Media004: Fe2+ (location: e)
rctn14: Fe2+ (location: c)


# Create a biomass equation

In [5]:
biomass_equation = PyFBA.metabolism.biomass_equation('gramnegative', modelseed.compounds)

biomass.py: No compound found for Fe2+ in our compounds dataset
biomass.py: No compound found for Fe3+ in our compounds dataset


_Remember_: Use `ctrl`+`/` to uncomment a line; `ctrl-A` `ctrl-/` to uncomment this whole block!

In [6]:
# sbml_bm_eqn = '(0.00778132482043096) cpd00063: Ca2 (location: c) +  (0.352889948968272) cpd00156: L-Valine (location: e) +  (0.00778132482043096) cpd00030: Mn2 (location: e) +  (0.00778132482043096) cpd00205: K (location: c) +  (0.428732289454499) cpd00035: L-Alanine (location: e) +  (0.128039715997337) cpd00060: L-Methionine (location: e) +  (0.15480760087483) cpd00066: L-Phenylalanine (location: c) +  (0.00778132482043096) cpd00017: S-Adenosyl-L-methionine (location: c) +  (0.00778132482043096) cpd00010: CoA (location: c) +  (0.0609084652443221) cpd15665: Peptidoglycan-polymer-n-subunits (location: c) +  (0.0841036156544863) cpd00052: CTP (location: c) +  (0.00778132482043096) cpd10516: fe3 (location: e) +  (0.01468498342018) cpd00357: TTP (location: c) +  (0.00778132482043096) cpd00099: Cl- (location: e) +  (0.01468498342018) cpd00356: dCTP (location: c) +  (0.00778132482043096) cpd10515: Fe2 (location: e) +  (0.00778132482043096) cpd00254: Mg (location: c) +  (0.242249358141304) cpd00322: L-Isoleucine (location: e) +  (0.00778132482043096) cpd00058: Cu2 (location: e) +  (0.00778132482043096) cpd00149: Co2 (location: c) +  (0.201205267995816) cpd00041: L-Aspartate (location: e) +  (1) cpd17043: RNA-transcription (location: c) +  (0.219496655995436) cpd00023: L-Glutamate (location: e) +  (0.219496655995436) cpd00053: L-Glutamine (location: e) +  (0.376088782528765) cpd00107: L-Leucine (location: e) +  (0.00778132482043096) cpd00220: Riboflavin (location: e) +  (0.179790960093822) cpd00054: L-Serine (location: e) +  (0.0472899299502361) cpd00065: L-Tryptophan (location: e) +  (0.0609084652443221) cpd02229: Bactoprenyl-diphosphate (location: c) +  (0.00778132482043096) cpd11493: ACP (location: c) +  (1) cpd17041: Protein-biosynthesis (location: c) +  (0.184698405654696) cpd00129: L-Proline (location: e) +  (0.135406821203723) cpd00038: GTP (location: c) +  (0.01468498342018) cpd00241: dGTP (location: c) +  (1) cpd17042: DNA-replication (location: c) +  (0.211466290532188) cpd00161: L-Threonine (location: e) +  (40.1101757365074) cpd00002: ATP (location: c) +  (0.00778132482043096) cpd00016: Pyridoxal-phosphate (location: c) +  (0.00778132482043096) cpd00048: Sulfate (location: e) +  (0.00778132482043096) cpd00003: NAD (location: c) +  (0.01468498342018) cpd00115: dATP (location: c) +  (0.115101904973216) cpd00069: L-Tyrosine (location: e) +  (0.00778132482043096) cpd00015: FAD (location: c) +  (0.201205267995816) cpd00132: L-Asparagine (location: e) +  (0.00778132482043096) cpd00006: NADP (location: c) +  (35.5386858537513) cpd00001: H2O (location: e) +  (0.0762884719008526) cpd00084: L-Cysteine (location: c) +  (0.0794113918032267) cpd00119: L-Histidine (location: e) +  (0.285970236774541) cpd00039: L-Lysine (location: e) +  (0.0908319049068452) cpd00062: UTP (location: c) +  (0.00778132482043096) cpd00034: Zn2 (location: e) +  (0.247156803702178) cpd00051: L-Arginine (location: e) +  (0.510820469745475) cpd00033: Glycine (location: e) >  (40) cpd00008: ADP (location: c) +  (39.9922186751796) cpd00009: Phosphate (location: e) +  (0.00778132482043096) cpd12370: apo-ACP (location: c) +  (1) cpd11416: Biomass (location: c) +  (40) cpd00067: H (location: e) +  (0.0609084652443221) cpd15666: Peptidoglycan-polymer-n-1-subunits (location: c) +  (0.405833094852252) cpd00012: PPi (location: e)'
# sbml_left_compounds = {'cpd00066: L-Phenylalanine (location: c)' : 0.15480760087483,  'cpd00016: Pyridoxal-phosphate (location: c)' : 0.00778132482043096,  'cpd00132: L-Asparagine (location: e)' : 0.201205267995816,  'cpd00156: L-Valine (location: e)' : 0.352889948968272,  'cpd00099: Cl- (location: e)' : 0.00778132482043096,  'cpd00038: GTP (location: c)' : 0.135406821203723,  'cpd00003: NAD (location: c)' : 0.00778132482043096,  'cpd17041: Protein-biosynthesis (location: c)' : 1.0,  'cpd00033: Glycine (location: e)' : 0.510820469745475,  'cpd00322: L-Isoleucine (location: e)' : 0.242249358141304,  'cpd00254: Mg (location: c)' : 0.00778132482043096,  'cpd17043: RNA-transcription (location: c)' : 1.0,  'cpd00048: Sulfate (location: e)' : 0.00778132482043096,  'cpd10515: Fe2 (location: e)' : 0.00778132482043096,  'cpd02229: Bactoprenyl-diphosphate (location: c)' : 0.0609084652443221,  'cpd11493: ACP (location: c)' : 0.00778132482043096,  'cpd00161: L-Threonine (location: e)' : 0.211466290532188,  'cpd00006: NADP (location: c)' : 0.00778132482043096,  'cpd00060: L-Methionine (location: e)' : 0.128039715997337,  'cpd00119: L-Histidine (location: e)' : 0.0794113918032267,  'cpd00052: CTP (location: c)' : 0.0841036156544863,  'cpd00051: L-Arginine (location: e)' : 0.247156803702178,  'cpd15665: Peptidoglycan-polymer-n-subunits (location: c)' : 0.0609084652443221,  'cpd00017: S-Adenosyl-L-methionine (location: c)' : 0.00778132482043096,  'cpd00030: Mn2 (location: e)' : 0.00778132482043096,  'cpd10516: fe3 (location: e)' : 0.00778132482043096,  'cpd00065: L-Tryptophan (location: e)' : 0.0472899299502361,  'cpd00084: L-Cysteine (location: c)' : 0.0762884719008526,  'cpd00023: L-Glutamate (location: e)' : 0.219496655995436,  'cpd17042: DNA-replication (location: c)' : 1.0,  'cpd00356: dCTP (location: c)' : 0.01468498342018,  'cpd00035: L-Alanine (location: e)' : 0.428732289454499,  'cpd00069: L-Tyrosine (location: e)' : 0.115101904973216,  'cpd00220: Riboflavin (location: e)' : 0.00778132482043096,  'cpd00129: L-Proline (location: e)' : 0.184698405654696,  'cpd00357: TTP (location: c)' : 0.01468498342018,  'cpd00205: K (location: c)' : 0.00778132482043096,  'cpd00149: Co2 (location: c)' : 0.00778132482043096,  'cpd00063: Ca2 (location: c)' : 0.00778132482043096,  'cpd00054: L-Serine (location: e)' : 0.179790960093822,  'cpd00001: H2O (location: e)' : 35.5386858537513,  'cpd00010: CoA (location: c)' : 0.00778132482043096,  'cpd00015: FAD (location: c)' : 0.00778132482043096,  'cpd00062: UTP (location: c)' : 0.0908319049068452,  'cpd00107: L-Leucine (location: e)' : 0.376088782528765,  'cpd00241: dGTP (location: c)' : 0.01468498342018,  'cpd00053: L-Glutamine (location: e)' : 0.219496655995436,  'cpd00039: L-Lysine (location: e)' : 0.285970236774541,  'cpd00034: Zn2 (location: e)' : 0.00778132482043096,  'cpd00058: Cu2 (location: e)' : 0.00778132482043096,  'cpd00002: ATP (location: c)' : 40.1101757365074,  'cpd00041: L-Aspartate (location: e)' : 0.201205267995816,  'cpd00115: dATP (location: c)' : 0.01468498342018}
# sbml_right_compounds = {'cpd00067: H (location: e)' : 40.0,  'cpd00012: PPi (location: e)' : 0.405833094852252,  'cpd00008: ADP (location: c)' : 40.0,  'cpd11416: Biomass (location: c)' : 1.0,  'cpd12370: apo-ACP (location: c)' : 0.00778132482043096,  'cpd00009: Phosphate (location: e)' : 39.9922186751796,  'cpd15666: Peptidoglycan-polymer-n-1-subunits (location: c)' : 0.0609084652443221}

# sbml_biomass = PyFBA.metabolism.Reaction('sbml_biomass', 'sbml_biomass')
# sbml_biomass.equation = sbml_bm_eqn
# parsecomp = re.compile('^(cpd\\d+): (.*?) \(location: (.)\)')
# for c in sbml_left_compounds:
#     m = parsecomp.match(c)
#     if not m:
#         sys.stderr.write(f"Can't parse {c}\n")
        
#     cpd = modelseed.get_compound_by_id(m.group(1))
#     if not cpd:
#         cpd = modelseed.get_compound_by_name(m.group(1))    
#     if not cpd:
#         print(f"Left: {m.group(1)} not found")
#         continue

#     if cpd.name != m.group(2):
#         sys.stderr.write(f"We found |{cpd.name}| for {m.group(1)} in the model seed, but have |{m.group(2)}| in the equation\n")
#     newcomp = PyFBA.metabolism.CompoundWithLocation.from_compound(cpd, m.group(3))
#     sbml_biomass.add_left_compounds({newcomp})
#     sbml_biomass.set_left_compound_abundance(newcomp, sbml_left_compounds[c])
        

# for c in sbml_right_compounds:
#     m = parsecomp.match(c)
#     if not m:
#         sys.stderr.write(f"Can't parse {c}\n")

#     cpd = modelseed.get_compound_by_id(m.group(1))
#     if not cpd:
#         cpd = modelseed.get_compound_by_name(m.group(1))    
#     if not cpd:
#         print(f"Left: {m.group(1)} not found")
#         continue
        
#     if cpd.name != m.group(2):
#         sys.stderr.write(f"We found |{cpd.name}| for {m.group(1)} in the model seed, but have |{m.group(2)}| in the equation\n")
#     newcomp = PyFBA.metabolism.CompoundWithLocation.from_compound(cpd, m.group(3))
#     sbml_biomass.add_right_compounds({newcomp})
#     sbml_biomass.set_right_compound_abundance(newcomp, sbml_right_compounds[c])


# Read the media and correct the names

In [7]:
# Read the media file
media = PyFBA.parse.read_media_file("/home/redwards/.local/lib/python3.9/site-packages/PyFBA-2.1-py3.9.egg/PyFBA/Biochemistry/media/ArgonneLB.txt")
# Correct the names
media = PyFBA.parse.correct_media_names(media, modelseed.compounds)
print(f"The media has {len(media)} compounds")

The media has 65 compounds


Checking media compounds: Our compounds do not include  Fe2+
Checking media compounds: Our compounds do not include  fe3
Please note: The warnings about media not being found in compounds are not fatal.
It just means that we did not find that compound anywhere in the reactions, and so it is unlikely to be
needed or used. We typically see a few of these in rich media.


# Read the reactions

In [8]:
reactions_to_run = set()
with open('rgood.txt', 'r') as f:
    for l in f:
        if l.startswith('rxn'):
            reactions_to_run.add(l.strip())
print(f"There are {len(reactions_to_run)} reactions to run")

There are 1399 reactions to run


# Add the biomass equation to the reactions to run

In [9]:
#reactions_to_run.add(sbml_biomass)

In [10]:
# for r in reactions_to_run:
#     if r not in modelseed.reactions:
#         sys.stderr.write(f"Error: reaction {r} not found\n")

# Run the FBA

In [11]:
uptake_secretion_reactions = set()

In [12]:
status, value, growth = PyFBA.fba.run_fba(modelseed.compounds, modelseed.reactions,
                                          reactions_to_run, media, biomass_equation,
                                          uptake_secretion_reactions, verbose=True)
print("The FBA completed with a flux value of {} --> growth: {}".format(value, growth))


create_stoichiometric_matrix is adding compound cpd00058: Cu2+ (location: e) from media to compounds
create_stoichiometric_matrix is adding compound Media004: Fe2+ (location: e) from media to compounds
create_stoichiometric_matrix is adding compound cpd00218: Niacin (location: e) from media to compounds
create_stoichiometric_matrix is adding compound cpd00393: Folate (location: e) from media to compounds
create_stoichiometric_matrix is adding compound cpd00001: H2O (location: e) from media to compounds
create_stoichiometric_matrix is adding compound cpd00149: Co2+ (location: e) from media to compounds
create_stoichiometric_matrix is adding compound cpd00067: H+ (location: e) from media to compounds
create_stoichiometric_matrix is adding compound cpd00156: L-Valine (location: e) from media to compounds
create_stoichiometric_matrix is adding compound cpd00182: Adenosine (location: e) from media to compounds
create_stoichiometric_matrix is adding compound cpd00030: Mn2+ (location: e) from

create_stoichiometric_matrix added compound cpd00067: H+ (location: c) from biomass to compounds
create_stoichiometric_matrix added compound cpd15666: Peptidoglycan polymer (n-1 subunits) (location: c) from biomass to compounds
create_stoichiometric_matrix added compound cpd12370: apo-ACP (location: c) from biomass to compounds
create_stoichiometric_matrix found 192 uptake and secretion reactions
In the model there are : 1325 compounds and 1592 reactions
We are loading 1325 rows and 1592 columns
Length of the media: 65
Number of reactions to run: 1399
Number of compounds in SM: 1325
Number of reactions in SM: 1592
Revised number of total reactions: 43966
Number of total compounds: 34117
SMat dimensions: 1325 x 1592


The FBA completed with a flux value of 0.0 --> growth: False


## Gapfilling the reactions

In [13]:
for c in biomass_equation.left_compounds:
    print(c)

cpd00002: ATP (location: c)
cpd00001: H2O (location: c)
cpd00149: Co2+ (location: c)
cpd00156: L-Valine (location: c)
cpd00062: UTP (location: c)
cpd00030: Mn2+ (location: c)
cpd00017: S-Adenosyl-L-methionine (location: c)
cpd00023: L-Glutamate (location: c)
cpd00107: L-Leucine (location: c)
rctn47: Fe3+ (location: c)
cpd11493: ACP (location: c)
cpd22528: CoA (location: c)
cpd00322: L-Isoleucine (location: c)
cpd15665: Peptidoglycan polymer (n subunits) (location: c)
cpd00099: Cl- (location: c)
cpd00016: Pyridoxal phosphate (location: c)
cpd00357: TTP (location: c)
cpd00356: dCTP (location: c)
cpd00035: L-Alanine (location: c)
cpd00052: CTP (location: c)
cpd00069: L-Tyrosine (location: c)
cpd00205: K+ (location: c)
cpd00132: L-Asparagine (location: c)
cpd17041: Protein biosynthesis (location: c)
cpd00038: GTP (location: c)
cpd00084: L-Cysteine (location: c)
cpd00033: Glycine (location: c)
cpd17042: DNA replication (location: c)
cpd00161: L-Threonine (location: c)
cpd00066: L-Phenylalan

In [14]:
for c in modelseed.compounds:
    if c.name == 'L-Threonine':
        print(c)

cpd00161: L-Threonine (location: c)
cpd00161: L-Threonine (location: e)
cpd00161: L-Threonine


## Comparing to the SBML file that does grow!

We pickled that data, and can load it here. 

In [15]:
sbml_filtered_compounds = pickle.load(open('compounds.pickle', 'rb'))
sbml_reactions = pickle.load(open('reactions.pickle', 'rb'))
sbml_reactions_to_run = pickle.load(open('reactions_to_run.pickle', 'rb'))
sbml_media = pickle.load(open('media.pickle', 'rb'))
sbml_biomass_equation = pickle.load(open('sbml_biomass.pickle', 'rb'))
sbml_uptake_secretion_reactions = pickle.load(open('uptake_secretion_reactions.pickle', 'rb'))

In [16]:
for r in sbml_reactions:
    for c in sbml_reactions[r].all_compounds():
        if '_' in c.name:
            print(c.name)

L_Alanine
L_Methionine
met_L_ala_L
L_Alanine
ala_L_Thr_L
L_Threonine
L_Aspartate
gly_asp_L
L_Cysteine
Gly_Cys
L_Glutamate
ala_L_glu_L
L_Alanine
gly_asn_L
L_Asparagine
Gly_Leu
L_Leucine
L_Alanine
L_alanylglycine
L_Alanine
L_Aspartate
ala_L_asp_L
L_Cysteine
Cys_Gly
Gly_Gln
L_Glutamine
Gly_Phe
L_Phenylalanine
L_Alanine
L_Histidine
Ala_His
L_Glutamate
gly_glu_L
L_Glutamine
L_Alanine
Ala_Gln
L_Tyrosine
Gly_Tyr
L_Proline
gly_pro_L
Ala_Leu
L_Leucine
L_Alanine
L_Methionine
Gly_Met
Glucose_1_phosphate
Crotonyl_CoA
Butyryl_CoA
Methacrylyl_CoA
Isobutyryl_CoA
2_Methylbutyryl_CoA
Tiglyl_CoA
D_Glucosamine1_phosphate
N_Acetyl_D_glucosamine1_phosphate
Acetyl_CoA
UDP_N_acetylglucosamine
N_Acetyl_D_glucosamine1_phosphate
L_Argininosuccinate
L_Arginine
3_oxoadipate_enol_lactone
4_Carboxymuconolactone
2_Demethylmenaquinol_8
2_Demethylmenaquinone_8
Ubiquinone_8
Ubiquinol_8
Menaquinol_8
Menaquinone_8
1_2_dianteisoheptadecanoyl_sn_glycerol_3_phosphate
1_anteisoheptadecanoyl_sn_glycerol_3_phosphate
1_isohexad

phosphatidylethanolamine_dioctadecanoyl
L_2_Lysophosphatidylethanolamine
2_Phospho_D_glycerate
L_Arginine
S_3_Hydroxyisobutyryl_CoA
Methacrylyl_CoA
S_3_Hydroxytetradecanoyl_CoA
2E_Tetradecenoyl_CoA
S_Hydroxydecanoyl_CoA
2E_Decenoyl_CoA
S_3_Hydroxydodecanoyl_CoA
2E_Dodecenoyl_CoA
Crotonyl_CoA
S_3_Hydroxybutyryl_CoA
2E_Hexenoyl_CoA
S_Hydroxyhexanoyl_CoA
2_methyl_3_hydroxy_butyryl_CoA
Tiglyl_CoA
2E_Hexadecenoyl_CoA
S_3_Hydroxyhexadecanoyl_CoA
S_Hydroxyoctanoyl_CoA
2E_Octenoyl_CoA
CDP_1_2_diisohexadecanoylglycerol
1_2_diisohexadecanoyl_sn_glycerol_3_phosphate
CDP_1_2_dioctadecanoylglycerol
1_2_dioctadecanoyl_sn_glycerol_3_phosphate
1_2_dianteisoheptadecanoyl_sn_glycerol_3_phosphate
CDP_1_2_dianteisoheptadecanoylglycerol
CDP_1_2_ditetradec_7_enoylglycerol
1_2_ditetradec_7_enoyl_sn_glycerol_3_phosphate
1_2_dihexadecanoyl_sn_glycerol_3_phosphate
CDP_1_2_dihexadecanoylglycerol
CDP_1_2_dioctadec_11_enoylglycerol
1_2_dioctadec_11_enoyl_sn_glycerol_3_phosphate
CDP_1_2_diisotetradecanoylglycerol
1

2_3_Cyclic_AMP
2_3_Cyclic_UMP
3_UMP
2_3_Cyclic_GMP
S_Adenosyl_L_methionine
Precorrin_3A
Precorrin_2
S_Adenosyl_homocysteine
Cobalt_precorrin_3
S_Adenosyl_L_methionine
Cobalt_precorrin_2
S_Adenosyl_homocysteine
S_Adenosyl_L_methionine
S_Adenosyl_homocysteine
Cobalt_precorrin_7
Cobalt_precorrin_8
Precorrin_6B
Precorrin_8
S_Adenosyl_L_methionine
S_Adenosyl_homocysteine
2_Demethylmenaquinol_8
2_Demethylmenaquinone_8
Menaquinol_8
Menaquinone_8
Ubiquinone_8
Ubiquinol_8
2_Demethylmenaquinol_8
1_4_Dihydroxy_2_naphthoate
Phytyl_diphosphate
1_4_Dihydroxy_2_naphthoate
5_Methylcytosine
ADP_L_glycero_D_manno_heptose
glucosyl_glucosyl_galactosyl_glucosyl_inner_core_oligosaccharide_lipid_A
core_oligosaccharide_lipid_A
N_Formimino_L_glutamate
4_Imidazolone_5_propanoate
D_fructose_6_phosphate
D_fructose_1_6_bisphosphate
Sedoheptulose7_phosphate
Sedoheptulose_1_7_bisphosphate
UDP_N_acetylglucosamine
Myristoyllipoteichoic_acid_n24__linked__N_acetyl_D_glucosamine
Myristoyllipoteichoic_acid_n24__linked__un

5_Phosphoribosyl_4_carboxy_5_aminoimidazole
L_Aspartate
5_10_Methenyltetrahydrofolate
10_Formyltetrahydrofolate
5_10_Methylenetetrahydrofolate
5_10_Methenyltetrahydrofolate
Glyceraldehyde3_phosphate
2_Keto_3_deoxy_6_phosphogluconate
4_Hydroxy_2_oxoglutarate
Propionyl_phosphate
Propionyl_CoA
Acetyl_CoA
cis_cis_Muconate
D_arabino_6_Phospho_hex_3_ulose
D_fructose_6_phosphate
2_Oxoglutarate
L_Glutamate
glucosyl_galactosyl_glucosyl_inner_core_oligosaccharide_lipid_A
UDP_glucose
galactosyl_glucosyl_inner_core_oligosaccharide_lipid_A
D_Ribulose5_phosphate
D_Arabinose5_phosphate
Acetyl_CoA
dTDP_4_amino_4_6_dideoxy_D_glucose
dTDP_4_acetamido_4_6_dideoxy_D_galactose
ribose_5_phosphate
Pseudouridine_5_phosphate
5_O__1_Carboxyvinyl_3_phosphoshikimate
2_Oxobutyrate
2_Aceto_2_hydroxybutanoate
2_Hydroxyethyl_ThPP
2_Hydroxyethyl_ThPP
S_Dihydroorotate
Ubiquinone_8
S_Dihydroorotate
Ubiquinol_8
Menaquinol_8
S_Dihydroorotate
Menaquinone_8
N2_Succinyl_L_ornithine
N2_Succinyl_L_arginine
2_C_methyl_D_erythri

L_Proline
D_Ribose
D_Ribose
L_Glutamine
L_Glutamine
gly_asp_L
gly_asp_L
D_Serine
D_Serine
D_Galactonate
D_Galactonate
D_Mannose
D_Mannose
Gly_Phe
Gly_Phe
L_Glutamate
L_Glutamate
L_Isoleucine
L_Isoleucine
D_Glucarate
D_Glucarate
L_Fucose_1_phosphate
L_Fucose_1_phosphate
met_L_ala_L
met_L_ala_L
5_Oxoproline
5_Oxoproline
L_Arginine
L_Arginine
Gly_Tyr
Gly_Tyr
gly_asn_L
gly_asn_L
L_Aspartate
L_Aspartate
D_Alanine
D_Alanine
L_Alanine
L_Alanine
D_Methionine
D_Methionine
L_Fucose
L_Fucose
L_Arabinose
L_Arabinose
Ala_His
Ala_His
L_Inositol
L_Inositol
L_Methionine_S_oxide
L_Methionine_S_oxide
L_Valine
L_Valine
L_Threonine
L_Threonine
Ala_Gln
Ala_Gln
L_Phenylalanine
L_Phenylalanine
Vitamin_B12r
Vitamin_B12r
L_Rhamnose
L_Rhamnose
L_Lysine
L_Lysine
D_glucose_6_phosphate
D_glucose_6_phosphate
Gly_Met
Gly_Met
Nicotinamide_ribonucleotide
Nicotinamide_ribonucleotide
R_3_Hydroxybutanoate
R_3_Hydroxybutanoate
Cl_
Cl_
D_mannose_6_phosphate
D_mannose_6_phosphate
N_Acetyl_D_glucosamine
N_Acetyl_D_glucosamin

So if we use the `sbml_reactions` and the `sbml_biomass_equation`, we get growth. Without those reactions we get an error. With the normal biomass equation, we don't get any growth

In [17]:
status, value, growth = PyFBA.fba.run_fba(modelseed.compounds, sbml_reactions,
                                          reactions_to_run, media, sbml_biomass_equation,
                                          set(), verbose=True)
print("The FBA completed with a flux value of {} --> growth: {}".format(value, growth))

create_stoichiometric_matrix added compound cpd00030: Mn2 (location: e) from biomass to compounds
create_stoichiometric_matrix added compound cpd00035: L_Alanine (location: e) from biomass to compounds
create_stoichiometric_matrix added compound cpd00132: L_Asparagine (location: e) from biomass to compounds
create_stoichiometric_matrix added compound cpd00107: L_Leucine (location: e) from biomass to compounds
create_stoichiometric_matrix added compound cpd00084: L_Cysteine (location: c) from biomass to compounds
create_stoichiometric_matrix added compound cpd00010: CoA (location: c) from biomass to compounds
create_stoichiometric_matrix added compound cpd00054: L_Serine (location: e) from biomass to compounds
create_stoichiometric_matrix added compound cpd00016: Pyridoxal_phosphate (location: c) from biomass to compounds
create_stoichiometric_matrix added compound cpd00053: L_Glutamine (location: e) from biomass to compounds
create_stoichiometric_matrix added compound cpd17042: DNA_rep

The FBA completed with a flux value of 400.4414741729346 --> growth: True


In [18]:
for c in modelseed.compounds:
    if c.id == 'cpd00060':
        print(c)

cpd00060: L-Methionine (location: c)
cpd00060: L-Methionine
cpd00060: L-Methionine (location: e)
cpd00060: L_Methionine (location: e)


### Note the normal biomass equation does not work!

We need to figure out what is missing!

In [19]:
biomass_equation = PyFBA.metabolism.biomass_equation('gram_negative')
status, value, growth = PyFBA.fba.run_fba(modelseed.compounds, sbml_reactions,
                                          reactions_to_run, media, biomass_equation,
                                          set(), verbose=True)
print("The FBA completed with a flux value of {} --> growth: {}".format(value, growth))

create_stoichiometric_matrix found 208 uptake and secretion reactions
In the model there are : 1232 compounds and 1608 reactions
We are loading 1232 rows and 1608 columns
Length of the media: 65
Number of reactions to run: 1399
Number of compounds in SM: 1232
Number of reactions in SM: 1608
Revised number of total reactions: 1782
Number of total compounds: 34157
SMat dimensions: 1232 x 1608


The FBA completed with a flux value of 0.0 --> growth: False


### And the modelseed.reactions gives us an error (or no growth

In [20]:
status, value, growth = PyFBA.fba.run_fba(modelseed.compounds, modelseed.reactions,
                                          reactions_to_run, media, sbml_biomass_equation,
                                          set(), verbose=True)
print("The FBA completed with a flux value of {} --> growth: {}".format(value, growth))

create_stoichiometric_matrix found 216 uptake and secretion reactions
In the model there are : 1360 compounds and 1616 reactions
We are loading 1360 rows and 1616 columns


The FBA completed with a flux value of -2.693908948286048e-13 --> growth: False


Length of the media: 65
Number of reactions to run: 1399
Number of compounds in SM: 1360
Number of reactions in SM: 1616
Revised number of total reactions: 43990
Number of total compounds: 34157
SMat dimensions: 1360 x 1616


In [21]:
for r in sbml_reactions:
    if r not in modelseed.reactions:
        print(r)

biomass_equation
EX_cpd00067
EX_cpd00609
EX_cpd00652
EX_cpd00041
EX_cpd00036
EX_cpd11590
EX_cpd11583
EX_cpd15603
EX_cpd15606
EX_cpd01017
EX_cpd11586
EX_cpd11589
EX_cpd11584
EX_cpd15604
EX_cpd11593
EX_cpd11587
EX_cpd11592
EX_cpd11580
EX_cpd11581
EX_cpd11585
EX_cpd11591
EX_cpd15605
EX_cpd11582
EX_cpd11588
EX_cpd15302
EX_cpd00156
EX_cpd00322
EX_cpd00107
EX_cpd01242
EX_cpd00105
EX_cpd00254
EX_cpd00149
EX_cpd11576
EX_cpd00060
EX_cpd01914
EX_cpd00637
EX_cpd00098
EX_cpd00971
EX_cpd00023
EX_cpd00396
EX_cpd03424
EX_cpd00423
EX_cpd00246
EX_cpd03279
EX_cpd00367
EX_cpd00277
EX_cpd00249
EX_cpd00184
EX_cpd00412
EX_cpd00182
EX_cpd00654
EX_cpd08023
EX_cpd09878
EX_cpd11578
EX_cpd00210
EX_cpd03048
EX_cpd11579
EX_cpd00395
EX_cpd11596
EX_cpd11575
EX_cpd00158
EX_cpd00009
EX_cpd04098
EX_cpd01048
EX_cpd00048
EX_cpd00106
EX_cpd00264
EX_cpd00118
EX_cpd08636
EX_cpd01171
EX_cpd00211
EX_cpd00142
EX_cpd00030
EX_cpd00129
EX_cpd00224
EX_cpd00154
EX_cpd00108
EX_cpd00550
EX_cpd00033
EX_cpd00054
EX_cpd00117
EX_cpd00080

In [22]:
tmpr = copy.deepcopy(modelseed.reactions)
tmpr['biomass_equation'] = sbml_reactions['biomass_equation']

In [23]:
status, value, growth = PyFBA.fba.run_fba(modelseed.compounds, tmpr,
                                          reactions_to_run, media, sbml_biomass_equation,
                                          set(), verbose=True)
print("The FBA completed with a flux value of {} --> growth: {}".format(value, growth))

create_stoichiometric_matrix found 216 uptake and secretion reactions
In the model there are : 1360 compounds and 1616 reactions
We are loading 1360 rows and 1616 columns


The FBA completed with a flux value of -2.693908948286048e-13 --> growth: False


Length of the media: 65
Number of reactions to run: 1399
Number of compounds in SM: 1360
Number of reactions in SM: 1616
Revised number of total reactions: 43991
Number of total compounds: 34157
SMat dimensions: 1360 x 1616


In [24]:
sbml_reactions['EX_cpd01022'].__dict__

{'id': 'EX_cpd01022',
 'model_seed_id': 'EX_cpd01022',
 'readable_name': 'EX_Lactate_e0',
 'description': None,
 'equation': ' (1.000000) cpd01022: Lactate (location: e) =  (1.000000) cpd01022: Lactate (location: e)',
 'direction': '=',
 'left_compounds': {<PyFBA.metabolism.compound.CompoundWithLocation at 0x7f1603c83c10>},
 'left_abundance': {<PyFBA.metabolism.compound.CompoundWithLocation at 0x7f1603c83c10>: 1.0},
 'right_compounds': {<PyFBA.metabolism.compound.CompoundWithLocation at 0x7f1603c83c40>},
 'right_abundance': {<PyFBA.metabolism.compound.CompoundWithLocation at 0x7f1603c83c40>: 1.0},
 'lower_bound': -1000.0,
 'upper_bound': 1000.0,
 'pLR': 0,
 'pRL': 0,
 'enzymes': set(),
 'ec_numbers': [],
 'pegs': set(),
 'deltaG_error': 0,
 'deltaG': 0,
 'inp': False,
 'outp': False,
 'is_transport': False,
 'ran': False,
 'is_biomass_reaction': False,
 'biomass_direction': False,
 'is_gapfilled': False,
 'gapfill_method': '',
 'is_uptake_secretion': False}

In [25]:
external = set()
for r in reactions_to_run:
    for c in modelseed.reactions[r].all_compounds():
        if c.location == 'e':
            external.add(c)
print(f"There are {len(external)} external compounds")

There are 172 external compounds


In [26]:
newr = {}
for e in external:
    n = PyFBA.metabolism.Reaction(f"EX_{e.id}", readable_name='external rctn {e.id}',
                                 equation = f"(1) {e} = (1) {e}", direction='=')
    n.add_left_compounds(c)
    n.add_right_compounds(c)
    n.set_left_compound_abundance(c, 1)
    n.set_right_compound_abundance(c, 1)
    n.lower_bound = -1000
    n.upper_bound = 1000
    newr[f"EX_{e.id}"] = n


In [27]:
mr = copy.deepcopy(modelseed.reactions)
mr.update(newr)
status, value, growth = PyFBA.fba.run_fba(modelseed.compounds, mr,
                                          reactions_to_run, media, sbml_biomass_equation,
                                          set(), verbose=True)
print("The FBA completed with a flux value of {} --> growth: {}".format(value, growth))

create_stoichiometric_matrix found 216 uptake and secretion reactions
In the model there are : 1360 compounds and 1616 reactions
We are loading 1360 rows and 1616 columns
Length of the media: 65
Number of reactions to run: 1399
Number of compounds in SM: 1360
Number of reactions in SM: 1616
Revised number of total reactions: 44162
Number of total compounds: 34157
SMat dimensions: 1360 x 1616


The FBA completed with a flux value of -2.693908948286048e-13 --> growth: False


In [28]:
mr['biomass_equation'] = sbml_reactions['biomass_equation']
status, value, growth = PyFBA.fba.run_fba(modelseed.compounds, mr,
                                          reactions_to_run, media, sbml_biomass_equation,
                                          set(), verbose=True)
print("The FBA completed with a flux value of {} --> growth: {}".format(value, growth))

create_stoichiometric_matrix found 216 uptake and secretion reactions
In the model there are : 1360 compounds and 1616 reactions
We are loading 1360 rows and 1616 columns
Length of the media: 65
Number of reactions to run: 1399
Number of compounds in SM: 1360
Number of reactions in SM: 1616
Revised number of total reactions: 44163
Number of total compounds: 34157
SMat dimensions: 1360 x 1616


The FBA completed with a flux value of -2.693908948286048e-13 --> growth: False


In [29]:
for r in sbml_reactions:
    if r not in mr:
        print(r)

EX_cpd15302
EX_cpd08636
EX_cpd02701
EX_cpd11416


In [30]:
sbml_reactions['EX_cpd08636'].__dict__

{'id': 'EX_cpd08636',
 'model_seed_id': 'EX_cpd08636',
 'readable_name': 'EX_4_5_dihydroxy_2_3_pentanedione_c0',
 'description': None,
 'equation': ' (1.000000) cpd08636: 4_5_dihydroxy_2_3_pentanedione (location: c) =  (1.000000) cpd08636: 4_5_dihydroxy_2_3_pentanedione (location: c)',
 'direction': '=',
 'left_compounds': {<PyFBA.metabolism.compound.CompoundWithLocation at 0x7f1603c9f400>},
 'left_abundance': {<PyFBA.metabolism.compound.CompoundWithLocation at 0x7f1603c9f400>: 1.0},
 'right_compounds': {<PyFBA.metabolism.compound.CompoundWithLocation at 0x7f1603c9f430>},
 'right_abundance': {<PyFBA.metabolism.compound.CompoundWithLocation at 0x7f1603c9f430>: 1.0},
 'lower_bound': -1000.0,
 'upper_bound': 1000.0,
 'pLR': 0,
 'pRL': 0,
 'enzymes': set(),
 'ec_numbers': [],
 'pegs': set(),
 'deltaG_error': 0,
 'deltaG': 0,
 'inp': False,
 'outp': False,
 'is_transport': False,
 'ran': False,
 'is_biomass_reaction': False,
 'biomass_direction': False,
 'is_gapfilled': False,
 'gapfill_met

In [31]:
sbml_partial = copy.deepcopy(sbml_reactions)
status, value, growth = PyFBA.fba.run_fba(modelseed.compounds, sbml_partial,
                                          reactions_to_run, media, sbml_biomass_equation,
                                          set(), verbose=True)
print("The FBA completed with a flux value of {} --> growth: {}".format(value, growth))

create_stoichiometric_matrix found 208 uptake and secretion reactions
In the model there are : 1187 compounds and 1608 reactions
We are loading 1187 rows and 1608 columns
Length of the media: 65
Number of reactions to run: 1399
Number of compounds in SM: 1187
Number of reactions in SM: 1608
Revised number of total reactions: 1782
Number of total compounds: 34157
SMat dimensions: 1187 x 1608


The FBA completed with a flux value of 400.4414741729346 --> growth: True


In [32]:
del sbml_partial['EX_cpd11416']

In [33]:
for f in ['EX_cpd15302', 'EX_cpd08636', 'EX_cpd02701']:
    del sbml_partial[f]

In [34]:
for r in sbml_partial:
    if r not in mr:
        print(r)

In [35]:
sbml_partial = copy.deepcopy(sbml_reactions)
status, value, growth = PyFBA.fba.run_fba(modelseed.compounds, sbml_partial,
                                          reactions_to_run, media, sbml_biomass_equation,
                                          set(), verbose=True)
print("The FBA completed with a flux value of {} --> growth: {}".format(value, growth))

create_stoichiometric_matrix found 208 uptake and secretion reactions
In the model there are : 1187 compounds and 1608 reactions
We are loading 1187 rows and 1608 columns
Length of the media: 65
Number of reactions to run: 1399
Number of compounds in SM: 1187
Number of reactions in SM: 1608
Revised number of total reactions: 1782
Number of total compounds: 34157
SMat dimensions: 1187 x 1608


The FBA completed with a flux value of 400.4414741729346 --> growth: True


In [36]:
for r in sbml_reactions:
    if r in modelseed.reactions:
        left_diff = 0
        right_diff = 0
        for l in modelseed.reactions[r].left_compounds:
            try:
                a = sbml_reactions[r].get_left_compound_abundance(l)
            except KeyError:
                left_diff += 1
        for l in modelseed.reactions[r].right_compounds:
            try:
                a = sbml_reactions[r].get_right_compound_abundance(l)
            except KeyError:
                right_diff += 1
        if left_diff > 0 or right_diff > 0:
            print(f"{r}\t{left_diff}\t{right_diff}")

rxn12636	3	2
rxn12644	2	2
rxn12637	2	2
rxn12844	2	2
rxn12640	2	2
rxn12645	2	2
rxn12845	2	2
rxn12641	2	2
rxn12633	2	2
rxn00650	2	2
rxn12646	2	2
rxn12846	2	2
rxn12642	2	2
rxn12634	2	2
rxn12639	2	2
rxn12847	2	2
rxn12638	2	2
rxn12643	2	2
rxn12635	2	2
rxn01675	1	1
rxn00868	2	3
rxn01923	2	2
rxn02268	2	2
rxn03638	2	2
rxn00293	1	2
rxn00802	1	2
rxn02483	2	2
rxn08976	2	1
rxn08975	2	1
rxn08971	2	1
rxn10215	1	2
rxn10219	1	2
rxn08089	3	1
rxn10212	3	1
rxn08083	3	1
rxn10214	1	2
rxn10211	2	1
rxn10218	1	2
rxn08086	3	1
rxn10217	1	2
rxn08087	3	1
rxn08088	3	1
rxn08085	3	1
rxn10216	1	2
rxn08084	3	1
rxn10213	2	1
rxn01199	1	2
rxn05572	2	2
rxn05565	2	2
rxn10206	1	1
rxn10203	2	1
rxn08549	3	1
rxn10209	1	1
rxn08551	3	1
rxn10205	1	1
rxn08552	3	1
rxn10204	1	1
rxn08547	2	1
rxn10208	1	1
rxn10210	1	1
rxn08548	3	1
rxn08546	2	1
rxn10202	3	1
rxn10207	1	1
rxn08550	3	1
rxn01674	1	1
rxn00369	1	1
rxn01370	1	1
rxn01706	1	1
rxn01516	1	1
rxn01515	1	1
rxn01510	1	1
rxn00715	1	1
rxn00712	1	1
rxn01128	1	1
rxn01679	1	1
rxn01129	1	1

rxn12510	1	2
rxn05176	1	3
rxn00161	1	2
rxn01857	1	2
rxn05231	0	1
rxn06076	3	2
rxn05233	0	1
rxn06075	3	2
rxn00808	1	2
rxn08900	2	1
rxn08901	2	1
rxn00935	1	0
rxn01477	1	2
rxn05496	2	2
rxn03435	1	2
rxn02187	0	1
rxn02186	1	1
rxn03436	1	1
rxn03068	1	2
rxn01790	0	2
rxn00584	1	1
rxn00085	1	3
rxn11702	2	2
rxn00251	2	2
rxn00295	1	1
rxn00214	1	1
rxn00265	1	1
rxn05317	1	2
rxn01368	3	2
rxn01800	3	2
rxn01219	1	0
rxn00364	1	0
rxn00707	1	1
rxn03514	2	3
rxn04048	2	2
rxn10160	2	2
rxn01127	1	0
rxn10052	1	1
rxn00097	1	0
rxn10181	1	1
rxn00291	2	2
rxn00340	2	3
rxn00772	1	2
rxn01987	2	1
rxn00410	1	2
rxn00412	2	3
rxn00544	0	1
rxn00011	2	3
rxn05680	1	2
rxn02792	3	3
rxn00350	1	2
rxn02791	2	2
rxn05221	2	2
rxn00568	3	3
rxn10182	1	1
rxn05687	1	2
rxn02405	0	3
rxn00395	2	1
rxn00171	0	2
rxn01000	1	2
rxn00675	0	2
rxn00175	1	3
rxn00986	2	2
rxn03630	3	2
rxn01867	2	2
rxn01868	2	2
rxn02305	3	2
rxn03384	1	1
rxn02288	1	1
rxn05551	1	2
rxn01507	1	2
rxn02400	2	2
rxn01670	2	2
rxn00363	1	2
rxn00708	1	2
rxn01218	1	2
rxn01521	1	2

In [37]:
for r in sbml_reactions:
    print(r)

rxn12636
rxn12644
rxn12637
rxn12844
rxn12640
rxn12645
rxn12845
rxn12641
rxn12633
rxn00650
rxn12646
rxn12846
rxn12642
rxn12634
rxn12639
rxn12847
rxn12638
rxn12643
rxn12635
rxn01675
rxn00868
rxn01923
rxn02268
rxn03638
rxn00293
rxn00802
rxn02483
rxn08976
rxn08975
rxn08971
rxn10215
rxn10219
rxn08089
rxn10212
rxn08083
rxn10214
rxn10211
rxn10218
rxn08086
rxn10217
rxn08087
rxn08088
rxn08085
rxn10216
rxn08084
rxn10213
rxn01199
rxn05572
rxn05565
rxn10206
rxn10203
rxn08549
rxn10209
rxn08551
rxn10205
rxn08552
rxn10204
rxn08547
rxn10208
rxn10210
rxn08548
rxn08546
rxn10202
rxn10207
rxn08550
rxn01674
rxn00369
rxn01370
rxn01706
rxn01516
rxn01515
rxn01510
rxn00715
rxn00712
rxn01128
rxn01679
rxn01129
rxn00709
rxn00368
rxn00365
rxn01705
rxn00713
rxn08954
rxn00225
rxn00985
rxn00541
rxn10155
rxn10157
rxn10770
rxn05536
rxn05544
rxn12848
rxn12851
rxn05539
rxn05541
rxn05537
rxn05543
rxn12849
rxn05533
rxn05540
rxn05534
rxn05547
rxn05546
rxn05542
rxn05535
rxn12850
rxn05545
rxn05538
rxn03422
rxn08941
rxn08943
r

rxn02831
rxn03136
rxn00800
rxn05165
rxn05181
rxn00216
rxn01169
rxn08194
rxn05029
rxn03535
rxn01093
rxn00988
rxn02346
rxn01990
rxn04082
rxn05617
rxn00606
rxn02373
rxn05581
rxn00726
rxn00727
rxn02319
rxn02177
rxn03147
rxn00095
rxn01211
rxn00907
rxn03884
rxn00328
rxn09180
rxn01602
rxn00686
rxn00670
rxn00173
rxn02782
rxn00371
rxn03644
rxn00539
rxn00184
rxn08619
rxn01241
rxn06493
rxn01522
rxn01297
rxn01117
rxn09289
rxn00776
rxn09296
rxn01255
rxn03194
rxn02185
rxn01360
rxn08335
rxn08336
rxn02927
rxn03958
rxn12500
rxn01322
rxn02007
rxn01280
rxn01281
rxn02287
rxn02774
rxn02475
rxn01304
rxn00147
rxn09167
rxn05312
rxn11268
rxn00646
rxn08708
rxn09240
rxn05156
rxn05151
rxn03886
rxn06673
rxn13784
rxn10184
rxn00245
rxn05571
rxn04704
rxn01629
rxn00552
rxn12146
rxn01123
rxn05313
rxn00966
rxn03084
rxn02342
rxn00292
rxn00297
rxn05147
rxn05173
rxn05466
rxn08707
rxn00927
rxn01299
rxn01226
rxn01545
rxn02173
rxn10161
rxn10162
rxn10163
rxn05155
rxn02476
rxn00745
rxn00781
rxn01331
rxn01133
rxn00973
rxn00642
r

In [38]:
sbml_reactions['rxn01516'].__dict__

{'id': 'rxn01516',
 'model_seed_id': 'rxn01516',
 'readable_name': 'dTTP_uridine_5_phosphotransferase_c0',
 'description': None,
 'equation': ' (1) cpd00357: TTP (location: c) +  (1) cpd00249: Uridine (location: e) =  (1) cpd00091: UMP (location: c) +  (1) cpd00297: dTDP (location: c)',
 'direction': '=',
 'left_compounds': {<PyFBA.metabolism.compound.CompoundWithLocation at 0x7f1604604e80>,
  <PyFBA.metabolism.compound.CompoundWithLocation at 0x7f1604604eb0>},
 'left_abundance': {<PyFBA.metabolism.compound.CompoundWithLocation at 0x7f1604604e80>: 1.0,
  <PyFBA.metabolism.compound.CompoundWithLocation at 0x7f1604604eb0>: 1.0},
 'right_compounds': {<PyFBA.metabolism.compound.CompoundWithLocation at 0x7f1604604f10>,
  <PyFBA.metabolism.compound.CompoundWithLocation at 0x7f1604604ee0>},
 'right_abundance': {<PyFBA.metabolism.compound.CompoundWithLocation at 0x7f1604604ee0>: 1.0,
  <PyFBA.metabolism.compound.CompoundWithLocation at 0x7f1604604f10>: 1.0},
 'lower_bound': -1000.0,
 'upper_bo

In [39]:
modelseed.reactions['rxn01516'].__dict__

{'id': 'rxn01516',
 'model_seed_id': 'rxn01516',
 'readable_name': "dTTP:uridine 5'-phosphotransferase",
 'description': None,
 'equation': '(1) Uridine[c] + (1) TTP[c] <=> (1) H+[c] + (1) UMP[c] + (1) dTDP[c]',
 'direction': '=',
 'left_compounds': {<PyFBA.metabolism.compound.CompoundWithLocation at 0x7f161acfbeb0>,
  <PyFBA.metabolism.compound.CompoundWithLocation at 0x7f161acfbf10>},
 'left_abundance': {<PyFBA.metabolism.compound.CompoundWithLocation at 0x7f161acfbeb0>: 1.0,
  <PyFBA.metabolism.compound.CompoundWithLocation at 0x7f161acfbf10>: 1.0},
 'right_compounds': {<PyFBA.metabolism.compound.CompoundWithLocation at 0x7f161acfbf40>,
  <PyFBA.metabolism.compound.CompoundWithLocation at 0x7f161acfbfa0>},
 'right_abundance': {<PyFBA.metabolism.compound.CompoundWithLocation at 0x7f161acfbf40>: 1.0,
  <PyFBA.metabolism.compound.CompoundWithLocation at 0x7f161acfbf70>: 1.0,
  <PyFBA.metabolism.compound.CompoundWithLocation at 0x7f161acfbfa0>: 1.0},
 'lower_bound': None,
 'upper_bound'

In [40]:
for r in reactions_to_run:
    for c in modelseed.reactions[r].left_compounds:
        print(f"left {c.id} abundance {0 - modelseed.reactions[r].get_left_compound_abundance(c)}")


left cpd00022 abundance -1.0
left cpd00033 abundance -1.0
left cpd00644 abundance -1.0
left cpd00971 abundance -1.0
left cpd00338 abundance -2.0
left cpd11437 abundance -1.0
left cpd00080 abundance -1.0
left cpd00727 abundance -1.0
left cpd00001 abundance -1.0
left cpd00102 abundance -1.0
left cpd00238 abundance -1.0
left cpd00037 abundance -24.0
left cpd15748 abundance -1.0
left cpd00003 abundance -1.0
left cpd00080 abundance -1.0
left cpd00002 abundance -1.0
left cpd01017 abundance -1.0
left cpd00001 abundance -1.0
left cpd00002 abundance -1.0
left cpd00001 abundance -1.0
left cpd00119 abundance -1.0
left cpd00002 abundance -1.0
left cpd02197 abundance -1.0
left cpd15523 abundance -1.0
left cpd00001 abundance -1.0
left cpd00001 abundance -1.0
left cpd11581 abundance -1.0
left cpd00031 abundance -1.0
left cpd00485 abundance -1.0
left cpd11420 abundance -1.0
left cpd00001 abundance -1.0
left cpd00533 abundance -1.0
left cpd00806 abundance -1.0
left cpd00002 abundance -1.0
left cpd00067

left cpd03587 abundance -1.0
left cpd00133 abundance -1.0
left cpd00001 abundance -1.0
left cpd00067 abundance -1.0
left cpd11533 abundance -1.0
left cpd00005 abundance -1.0
left cpd00009 abundance -1.0
left cpd00003 abundance -1.0
left cpd00102 abundance -1.0
left cpd15418 abundance -1.0
left cpd00080 abundance -1.0
left cpd15684 abundance -1.0
left cpd00080 abundance -1.0
left cpd11492 abundance -1.0
left cpd11515 abundance -1.0
left cpd00153 abundance -1.0
left cpd00004 abundance -1.0
left cpd00067 abundance -1.0
left cpd00007 abundance -1.0
left cpd00113 abundance -1.0
left cpd00025 abundance -2.0
left cpd00067 abundance -1.0
left cpd00003 abundance -1.0
left cpd00635 abundance -2.0
left cpd00067 abundance -1.0
left cpd10596 abundance -1.0
left cpd00449 abundance -1.0
left cpd00003 abundance -1.0
left cpd00161 abundance -1.0
left cpd00067 abundance -1.0
left cpd00002 abundance -1.0
left cpd00001 abundance -1.0
left cpd15605 abundance -1.0
left cpd00067 abundance -1.0
left cpd09027 

left cpd11432 abundance -1.0
left cpd15671 abundance -1.0
left cpd03833 abundance -1.0
left cpd00017 abundance -1.0
left cpd03116 abundance -1.0
left cpd11501 abundance -1.0
left cpd00209 abundance -1.0
left cpd15499 abundance -1.0
left cpd00067 abundance -1.0
left cpd00002 abundance -1.0
left cpd00010 abundance -1.0
left cpd01772 abundance -1.0
left cpd00047 abundance -1.0
left cpd00067 abundance -1.0
left cpd00042 abundance -2.0
left cpd00006 abundance -1.0
left cpd00067 abundance -2.0
left cpd00007 abundance -1.0
left cpd02083 abundance -1.0
left cpd00002 abundance -1.0
left cpd00001 abundance -1.0
left cpd11583 abundance -1.0
left cpd00035 abundance -1.0
left cpd00216 abundance -1.0
left cpd00013 abundance -1.0
left cpd00221 abundance -1.0
left cpd15560 abundance -1.0
left cpd02210 abundance -1.0
left cpd00054 abundance -1.0
left cpd00009 abundance -1.0
left cpd03279 abundance -1.0
left cpd00002 abundance -1.0
left cpd00179 abundance -1.0
left cpd00001 abundance -1.0
left cpd00277 

left cpd00067 abundance -1.0
left cpd08301 abundance -1.0
left cpd00067 abundance -1.0
left cpd00005 abundance -1.0
left cpd11537 abundance -1.0
left cpd00001 abundance -1.0
left cpd11592 abundance -1.0
left cpd00002 abundance -1.0
left cpd00069 abundance -1.0
left cpd00084 abundance -1.0
left cpd08289 abundance -1.0
left cpd00001 abundance -1.0
left cpd00003 abundance -1.0
left cpd00114 abundance -1.0
left cpd00067 abundance -1.0
left cpd00130 abundance -1.0
left cpd00010 abundance -1.0
left cpd02693 abundance -1.0
left cpd00003 abundance -1.0
left cpd00002 abundance -1.0
left cpd00041 abundance -1.0
left cpd02893 abundance -1.0
left cpd00009 abundance -1.0
left cpd00008 abundance -1.0
left cpd00067 abundance -4.0
left cpd08625 abundance -1.0
left cpd00001 abundance -1.0
left cpd00219 abundance -1.0
left cpd00006 abundance -1.0
left cpd15753 abundance -1.0
left cpd00037 abundance -24.0
left cpd00100 abundance -1.0
left cpd00003 abundance -1.0
left cpd03048 abundance -1.0
left cpd00007

left cpd00449 abundance -1.0
left cpd00842 abundance -1.0
left cpd00003 abundance -1.0
left cpd00061 abundance -1.0
left cpd00158 abundance -1.0
left cpd00067 abundance -1.0
left cpd00308 abundance -1.0
left cpd00002 abundance -1.0
left cpd00001 abundance -1.0
left cpd10516 abundance -1.0
left cpd00002 abundance -1.0
left cpd04920 abundance -1.0
left cpd00004 abundance -1.0
left cpd00067 abundance -1.0
left cpd15500 abundance -1.0
left cpd00120 abundance -1.0
left cpd00003 abundance -1.0
left cpd15560 abundance -1.0
left cpd00001 abundance -1.0
left cpd00027 abundance -1.0
left cpd11464 abundance -1.0
left cpd00003 abundance -1.0
left cpd00062 abundance -1.0
left cpd02611 abundance -1.0
left cpd02501 abundance -1.0
left cpd00007 abundance -1.0
left cpd00002 abundance -1.0
left cpd00168 abundance -1.0
left cpd02720 abundance -1.0
left cpd00006 abundance -1.0
left cpd00179 abundance -1.0
left cpd00002 abundance -1.0
left cpd01262 abundance -1.0
left cpd00001 abundance -1.0
left cpd00971 

left cpd02826 abundance -1.0
left cpd00117 abundance -1.0
left cpd00001 abundance -1.0
left cpd00015 abundance -1.0
left cpd00002 abundance -1.0
left cpd00027 abundance -1.0
left cpd00001 abundance -1.0
left cpd00002 abundance -1.0
left cpd08023 abundance -1.0
left cpd00001 abundance -1.0
left cpd01620 abundance -1.0
left cpd00003 abundance -1.0
left cpd00001 abundance -1.0
left cpd15535 abundance -1.0
left cpd02698 abundance -1.0
left cpd00001 abundance -1.0
left cpd00004 abundance -1.0
left cpd00067 abundance -1.0
left cpd11560 abundance -1.0
left cpd00608 abundance -1.0
left cpd00345 abundance -1.0
left cpd00003 abundance -1.0
left cpd11492 abundance -1.0
left cpd11557 abundance -1.0
left cpd00515 abundance -1.0
left cpd00067 abundance -1.0
left cpd00668 abundance -1.0
left cpd00005 abundance -1.0
left cpd00132 abundance -1.0
left cpd00067 abundance -1.0
left cpd00179 abundance -1.0
left cpd00061 abundance -1.0
left cpd00001 abundance -1.0
left cpd15526 abundance -1.0
left cpd00179 

left cpd00064 abundance -1.0
left cpd00002 abundance -1.0
left cpd00220 abundance -1.0
left cpd00117 abundance -1.0
left cpd00067 abundance -1.0
left cpd00002 abundance -1.0
left cpd00805 abundance -1.0
left cpd00002 abundance -1.0
left cpd00001 abundance -1.0
left cpd00531 abundance -1.0
left cpd00102 abundance -1.0
left cpd00072 abundance -1.0
left cpd15721 abundance -1.0
left cpd00001 abundance -1.0
left cpd00223 abundance -1.0
left cpd00006 abundance -1.0
left cpd00022 abundance -1.0
left cpd01695 abundance -1.0
left cpd03448 abundance -1.0
left cpd00067 abundance -3.0
left cpd00005 abundance -1.0
left cpd00007 abundance -1.0
left cpd00348 abundance -1.0
left cpd00062 abundance -1.0
left cpd00002 abundance -1.0
left cpd00011 abundance -1.0
left cpd00013 abundance -1.0
left cpd00002 abundance -1.0
left cpd00031 abundance -1.0
left cpd00002 abundance -1.0
left cpd00001 abundance -1.0
left cpd00058 abundance -1.0
left cpd00002 abundance -1.0
left cpd00001 abundance -1.0
left cpd11593 

left cpd00004 abundance -1.0
left cpd00067 abundance -1.0
left cpd11531 abundance -1.0
left cpd00307 abundance -1.0
left cpd00067 abundance -1.0
left cpd00067 abundance -2.0
left cpd00005 abundance -1.0
left cpd08372 abundance -1.0
left cpd00027 abundance -1.0
left cpd00055 abundance -1.0
left cpd00171 abundance -1.0
left cpd00067 abundance -1.0
left cpd15328 abundance -1.0
left cpd11825 abundance -1.0
left cpd15500 abundance -1.0
left cpd00080 abundance -1.0
left cpd15560 abundance -1.0
left cpd16654 abundance -1.0
left cpd00001 abundance -1.0
left cpd15524 abundance -1.0
left cpd00201 abundance -1.0
left cpd02851 abundance -1.0
left cpd03831 abundance -1.0
left cpd15485 abundance -1.0
left cpd02536 abundance -1.0
left cpd00286 abundance -1.0
left cpd02968 abundance -1.0
left cpd00022 abundance -1.0
left cpd00047 abundance -1.0
left cpd00001 abundance -1.0
left cpd00018 abundance -1.0
left cpd00002 abundance -1.0
left cpd00655 abundance -1.0
left cpd15417 abundance -1.0
left cpd00080 

left cpd00067 abundance -1.0
left cpd11466 abundance -1.0
left cpd00010 abundance -1.0
left cpd15523 abundance -1.0
left cpd00052 abundance -1.0
left cpd00051 abundance -1.0
left cpd00064 abundance -1.0
left cpd00009 abundance -1.0
left cpd03471 abundance -1.0
left cpd03583 abundance -1.0
left cpd11522 abundance -1.0
left cpd00002 abundance -1.0
left cpd00072 abundance -1.0
left cpd00001 abundance -1.0
left cpd03411 abundance -1.0
left cpd00371 abundance -1.0
left cpd00003 abundance -1.0
left cpd00010 abundance -1.0
left cpd00001 abundance -1.0
left cpd00003 abundance -1.0
left cpd01324 abundance -1.0
left cpd00002 abundance -1.0
left cpd15308 abundance -1.0
left cpd00002 abundance -1.0
left cpd00001 abundance -1.0
left cpd00395 abundance -1.0
left cpd00971 abundance -1.0
left cpd00023 abundance -1.0
left cpd11484 abundance -1.0
left cpd00387 abundance -1.0
left cpd15302 abundance -1.0
left cpd00001 abundance -1.0
left cpd15528 abundance -1.0
left cpd00012 abundance -1.0
left cpd00873 

In [41]:
modelseed.get_compound_by_id('cpd00037').name

'UDP-N-acetylglucosamine'

In [42]:
for c in modelseed.compounds:
    if c.id == 'cpd00060':
        if isinstance(c, PyFBA.metabolism.CompoundWithLocation):
            print(f"{c} ({c.id}) ({c.name}) ({c.location}): {c.__hash__()}")
            print(f"{hash((c.id, c.name, c.location))}")
        else:
            print(f"{c} ({c.id}) ({c.name}): {c.__hash__()}")

cpd00060: L-Methionine (location: c) (cpd00060) (L-Methionine) (c): -3751590082554226200
-391124026151041407
cpd00060: L-Methionine (cpd00060) (L-Methionine): -4479078554417336977
cpd00060: L-Methionine (location: e) (cpd00060) (L-Methionine) (e): -1698983126409485585
-2736874452990668920
cpd00060: L_Methionine (location: e) (cpd00060) (L_Methionine) (e): -4752263916683945563
-2988361088793910848


In [43]:
cid = 'cpd00060'
cname = 'L_Methionine'
cname2 = 'L-Methionine'
cloc = 'e'
for i in range(5):
    print(f"{hash((cid, cname, cloc))}")
    print(f"{hash((cid, cname2, cloc))}")

-2988361088793910848
-2736874452990668920
-2988361088793910848
-2736874452990668920
-2988361088793910848
-2736874452990668920
-2988361088793910848
-2736874452990668920
-2988361088793910848
-2736874452990668920
